# The 3D Knapsack Problem

### - Problem Configuration:

 ![title](img/input_output.png)

### - Challenge Set Up

**Knapsack Dimension:** 

In [13]:
knapsack_dim = 5
knapsack = (H = 10, W = 20, L = 10)

(H = 10, W = 20, L = 10)

**List of items:**

In [16]:
items = [(vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (vi = 105, dim = (ai = 3, bi = 3, ci = 3)),
         (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)),
         (vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3))]

6-element Array{NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}},1}:
 (vi = 10, dim = (ai = 10, bi = 3, ci = 5))
 (vi = 105, dim = (ai = 3, bi = 3, ci = 3))
 (vi = 20, dim = (ai = 3, bi = 3, ci = 10))
 (vi = 95, dim = (ai = 3, bi = 3, ci = 3)) 
 (vi = 30, dim = (ai = 1, bi = 3, ci = 5)) 
 (vi = 85, dim = (ai = 3, bi = 3, ci = 3)) 

**Set of Selected Items ⟹ J:** 

In [3]:
item_zero = (vi = 0, dim = (ai = 0, bi = 0, ci = 0))
J = [(item_zero, (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))]

1-element Array{Tuple{NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}},NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},NamedTuple{(:hi, :wi),Tuple{String,String}}},1}:
 ((vi = 0, dim = (ai = 0, bi = 0, ci = 0)), (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))

In [4]:
V = 0

0

# Placement Strategy:
## - Thinking in Corners

![title](img/empty.png)

 ![title](img/one.png)

 ![title](img/many_items.png)

### - Implementation

**List of Possible Corners:** 

In [ ]:
list_of_possible_corners = [(xi = 0, yi = 0, zi = 0)]

### Adding an Item to the Knapsack then consists of: 

In [7]:
function add_to_knapsack(item, J, knapsack, list_of_possible_corners)

    for i in 1:length(list_of_possible_corners)
        corner = list_of_possible_corners[i]
        corner_space = available_space(corner, knapsack)
        enough, relations = is_enough_space(corner_space, item)
        if enough
            ordered_space_dims = order_relations_in_xyz(relations)
            hi_wi_dims = get_hi_dim_and_wi_dim(relations)
            push!(J,(item, (xi = corner.xi, yi = corner.yi, zi = corner.zi), hi_wi_dims))
            remove!(list_of_possible_corners, corner)
            list_of_possible_corners = add_candidate_corners(list_of_possible_corners, item, corner, hi_wi_dims)
            return list_of_possible_corners
        end
    end
    return list_of_possible_corners
end

add_to_knapsack (generic function with 1 method)

### Now, The Utility Functions: 

In [8]:
function order_items_by(items, key)
    if key == "value"
        print(items[1].vi)
        valuable_items = sort(items, rev=true, by = x -> x[1])
        print(valuable_items)
        return valuable_items
    else
        heaviest_items = sort(items, rev=true, by = x -> x[2].ai * x[2].bi * x[2].ci)
        print(heaviest_items)
        return heaviest_items
    end
end

function split_in_halves(items)
    if mod(length(items), 2) == 0
        n = convert(Int, length(items) / 2) 
    else
        n = convert(Int, length(items) / 2 + 0.5)
    end
    first_half = [items[i] for i in 1:n]
    second_half = [items[i] for i in n+1:length(items)]

    return first_half, second_half
end

function get_largest_to_smallest_dim_list(a, b, c)
    dim = [a, b, c]
    dim_sorted = sort(dim)
    g1 = pop!(dim_sorted)
    g2 = pop!(dim_sorted)
    g3 = pop!(dim_sorted)
    g1_idx = findall(x->x==g1, dim)[1]
    dim[g1_idx] = -1
    g2_idx = findall(x->x==g2, dim)[1]
    dim[g2_idx] = -1
    g3_idx = findall(x->x==g3, dim)[1]
    
    largest_to_smallest_dim_list = [(g1_idx, g1), (g2_idx, g2), (g3_idx, g3)]

    return largest_to_smallest_dim_list
    
end

function available_space(corner, knapsack)
    return (x = knapsack.W - corner.xi, y = knapsack.L - corner.yi, z = knapsack.H - corner.zi)
end

function is_enough_space(corner_space, item)
    item_dims = get_largest_to_smallest_dim_list(item[2].ai, item[2].bi, item[2].ci)
    space_dims = get_largest_to_smallest_dim_list(corner_space.x, corner_space.y, corner_space.z)
    enough = true
    relations = [((0,0),(0,0))]

    for i in 1:3
        space = space_dims[i][2] - item_dims[i][2]
        if space < 0
            relation = nothing
            enough = false
            break
        end
        push!(relations, (space_dims[i], item_dims[i]))
    end
    popfirst!(relations)
    return enough, relations
end

function order_relations_in_xyz(relations)
    return sort(relations, by = x -> x[1][1][1])
end

function get_hi_dim_and_wi_dim(relations)
    ordered_relations = order_relations_in_xyz(relations)
    dim_dict = Dict(1 => "ai", 2 => "bi", 3 => "ci")
    (hi = dim_dict[ordered_relations[3][2][1]], wi = dim_dict[ordered_relations[1][2][1]])
end

function add3DTuple(tuple1, tuple2)
    partial_results = [0]

    for i in 1:length(tuple1)
        push!(partial_results, tuple1[i] + tuple2[i])
    end

    return new_tuple = (partial_results[2], partial_results[3], partial_results[4])
end

function remove!(a, item)
    deleteat!(a, findall(x->x==item, a))
end

function item_actual_orientation(item_dim, orientation)
    dim_dict = Dict("ai" => 1, "bi" => 2, "ci" => 3)
    list_dim = [1, 2, 3]
    z = orientation[1]
    x = orientation[2]
    item_x = item_dim[dim_dict[x]]
    remove!(list_dim, dim_dict[x])
    item_z = item_dim[dim_dict[z]]
    remove!(list_dim, dim_dict[z])
    item_y = item_dim[list_dim[1]]

    return (item_x, item_y, item_z)
end

function add_candidate_corners(conrner_list, item, coordinates, orientation)
    oriented_item = item_actual_orientation(item.dim, orientation)
        
    new_corner_1 = add3DTuple(coordinates, (oriented_item[1], 0, 0))
    push!(conrner_list, (xi = new_corner_1[1], yi = new_corner_1[2], zi = new_corner_1[3]))
    new_corner_2 = add3DTuple(coordinates, (0, oriented_item[2], 0))
    push!(conrner_list, (xi = new_corner_2[1], yi = new_corner_2[2], zi = new_corner_2[3]))
    new_corner_3 = add3DTuple(coordinates, (0, 0, oriented_item[3]))
    push!(conrner_list, (xi = new_corner_3[1], yi = new_corner_3[2], zi = new_corner_3[3]))

    return conrner_list
end

function calculate_V_in_J(J)
    V = 0
    for i in 1:length(J)
        V += J[i][1][1]
    end
    return V
end

calculate_V_in_J (generic function with 1 method)

## Strategies:  

 ### - Strategy 1 [Putting Rock first then later Sand]
        - Step1: Order them based on Weight
        - Step2: Divide them in half
        - Step3: Order each half based on Value
        - Step4: Take the heaviest and most valuable until it all fits
        - Step5: Take the lightest and most valuable if there is enough space left

  ### - Strategy 2 [Alternate between Rock and Sand]
        - Step1: Order them based on Weight
        - Step2: Divide them in half
        - Step3: Order each half based on Value
        - Step4: Take one of the heaviest and most valuable 
        - Step5: Take one of the lightest and most valuable
        - Step6: Repeat until there is no space left

  ### - Strategy 3
        - Step1: Order them based on Weight
        - Step2: Divide them in half
        - Step3: Order each half based on Value
        - Step4: Take one of the lightest and most valuable until it all fits
        - Step5: Take one of the heaviest and most valuable if there is enough space left

  ### - Strategy 4
        - Step1: Order them based on Value
        - Step2: Divide them in half
        - Step3: Order each half based on Weight
        - Step4: Take the heaviest and most valuable until it all fits
        - Step5: Take the lightest and most valuable if there is enough space left

  ### - Strategy 5
        - Step1: Order them based on Value
        - Step2: Divide them in half
        - Step3: Order each half based on Weight
        - Step4: Take one of the heaviest and most valuable 
        - Step5: Take one of the lightest and most valuable
        - Step6: Repeat until there is no space left

  ### - Strategy 6
        - Step1: Order them based on Value
        - Step2: Divide them in half
        - Step3: Order each half based on Weight
        - Step4: Take one of the lightest and most valuable until it all fits
        - Step5: Take one of the heaviest and most valuable if there is enough space left

  ### - Strategy 7 [Just Greedy]
        - Step1: Order them based on Value
        - Step2: Take one by one from the most valuable to the least until there is no space left

### Restarting inicial variables:

In [17]:
item_zero = (vi = 0, dim = (ai = 0, bi = 0, ci = 0))
J = [(item_zero, (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))]
list_of_possible_corners = [(xi = 0, yi = 0, zi = 0)]

1-element Array{NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},1}:
 (xi = 0, yi = 0, zi = 0)

In [18]:
# - Strategy 1 [Just Greedy]
#    Step1: Order them based on Value
items_most_valuable_to_least = order_items_by(items, "value")
#    Step2: Take one by one from the most valuable to the least until there is no space left
for i in 1:length(items_most_valuable_to_least)
    add_to_knapsack(items_most_valuable_to_least[i], J, knapsack, list_of_possible_corners)
end

10NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 10, dim = (ai = 10, bi = 3, ci = 5))]

**Output:** 

In [19]:
J

7-element Array{Tuple{NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}},NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},NamedTuple{(:hi, :wi),Tuple{String,String}}},1}:
 ((vi = 0, dim = (ai = 0, bi = 0, ci = 0)), (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))  
 ((vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 0, zi = 0), (hi = "ci", wi = "ai"))
 ((vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (xi = 3, yi = 0, zi = 0), (hi = "ci", wi = "ai")) 
 ((vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 3, zi = 0), (hi = "bi", wi = "ai")) 
 ((vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (xi = 0, yi = 0, zi = 3), (hi = "ai", wi = "ci")) 
 ((vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (xi = 6, yi = 0, zi = 0), (hi = "bi", wi = "ci"))
 ((vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (xi = 3, yi = 3, zi = 0), (hi = "ci", wi = "ai"))

In [20]:
V = calculate_V_in_J(J)

345

### Restarting inicial variables:

In [21]:
item_zero = (vi = 0, dim = (ai = 0, bi = 0, ci = 0))
J = [(item_zero, (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))]
list_of_possible_corners = [(xi = 0, yi = 0, zi = 0)]

1-element Array{NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},1}:
 (xi = 0, yi = 0, zi = 0)

In [22]:
#- Strategy 2 [Putting Rock first then later Sand]
#    Step1: Order them based on Value
items_most_valuable_to_least =  order_items_by(items, "value")
#    Step2: Divide them in half
valuable_half, less_valuable_half = split_in_halves(items_most_valuable_to_least)
#    Step3: Order each half based on Volume
valuable_and_heavy = order_items_by(valuable_half, "volume")
less_valuable_and_heavy = order_items_by(less_valuable_half, "volume")
#    Step4: Take the heaviest and most valuable until it all fits
for i in 1:length(valuable_and_heavy)
    add_to_knapsack(valuable_and_heavy[i], J, knapsack, list_of_possible_corners)
end
#    Step5: Take the lightest and most valuable if there is enough space left
for i in 1:length(less_valuable_and_heavy)
    add_to_knapsack(less_valuable_and_heavy[i], J, knapsack, list_of_possible_corners)
end

10NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 10, dim = (ai = 10, bi = 3, ci = 5))]NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3))]NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5))]

**Output:** 

In [23]:
J

7-element Array{Tuple{NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}},NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},NamedTuple{(:hi, :wi),Tuple{String,String}}},1}:
 ((vi = 0, dim = (ai = 0, bi = 0, ci = 0)), (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))  
 ((vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 0, zi = 0), (hi = "ci", wi = "ai"))
 ((vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (xi = 3, yi = 0, zi = 0), (hi = "ci", wi = "ai")) 
 ((vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 3, zi = 0), (hi = "bi", wi = "ai")) 
 ((vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (xi = 0, yi = 0, zi = 3), (hi = "bi", wi = "ai"))
 ((vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (xi = 6, yi = 0, zi = 0), (hi = "bi", wi = "ci"))
 ((vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (xi = 3, yi = 3, zi = 0), (hi = "bi", wi = "ci")) 

In [24]:
V = calculate_V_in_J(J)

345

### Restarting inicial variables:

In [25]:
item_zero = (vi = 0, dim = (ai = 0, bi = 0, ci = 0))
J = [(item_zero, (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))]
list_of_possible_corners = [(xi = 0, yi = 0, zi = 0)]

1-element Array{NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},1}:
 (xi = 0, yi = 0, zi = 0)

In [26]:
#- Strategy 3 [Alternate between Rock and Sand]
#    Step1: Order them based on Value
items_most_valuable_to_least =  order_items_by(items, "value")
#    Step2: Divide them in half
valuable_half, less_valuable_half = split_in_halves(items_most_valuable_to_least)
#    Step3: Order each half based on Volume
valuable_and_heavy = order_items_by(valuable_half, "volume")
less_valuable_and_heavy = order_items_by(less_valuable_half, "volume")
#    Step4: Repeat until there is no space left
for i in 1:length(less_valuable_and_heavy)
    #    Step5: Take one of the heaviest and most valuable 
    add_to_knapsack(valuable_and_heavy[i], J, knapsack, list_of_possible_corners)
    #    Step6: Take one of the lightest and most valuable
    add_to_knapsack(less_valuable_and_heavy[i], J, knapsack, list_of_possible_corners)
end
if length(less_valuable_and_heavy) < length(valuable_and_heavy)
    add_to_knapsack(valuable_and_heavy[length(valuable_and_heavy)], J, knapsack, list_of_possible_corners)
end

10NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 10, dim = (ai = 10, bi = 3, ci = 5))]NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3))]NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5))]

**Output:** 

In [27]:
J

7-element Array{Tuple{NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}},NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},NamedTuple{(:hi, :wi),Tuple{String,String}}},1}:
 ((vi = 0, dim = (ai = 0, bi = 0, ci = 0)), (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))  
 ((vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 0, zi = 0), (hi = "ci", wi = "ai"))
 ((vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (xi = 3, yi = 0, zi = 0), (hi = "bi", wi = "ai"))
 ((vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 3, zi = 0), (hi = "bi", wi = "ai")) 
 ((vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (xi = 0, yi = 0, zi = 3), (hi = "bi", wi = "ci"))
 ((vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (xi = 13, yi = 0, zi = 0), (hi = "bi", wi = "ci"))
 ((vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (xi = 3, yi = 5, zi = 0), (hi = "bi", wi = "ci")) 

In [29]:
V = calculate_V_in_J(J)

0

### Restarting inicial variables:

In [28]:
item_zero = (vi = 0, dim = (ai = 0, bi = 0, ci = 0))
J = [(item_zero, (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))]
list_of_possible_corners = [(xi = 0, yi = 0, zi = 0)]

1-element Array{NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},1}:
 (xi = 0, yi = 0, zi = 0)

In [30]:
#- Strategy 4 [Putting Sand first then later Rock]
#    Step1: Order them based on Value
items_most_valuable_to_least =  order_items_by(items, "value")
#    Step2: Divide them in half
valuable_half, less_valuable_half = split_in_halves(items_most_valuable_to_least)
#    Step3: Order each half based on Volume
valuable_and_heavy = order_items_by(valuable_half, "volume")
less_valuable_and_heavy = order_items_by(less_valuable_half, "volume")
#    Step4: Take one of the lightest and most valuable until it all fits
for i in 1:length(less_valuable_and_heavy)
    add_to_knapsack(less_valuable_and_heavy[i], J, knapsack, list_of_possible_corners)
end
#    Step5: Take one of the heaviest and most valuable if there is enough space left
for i in 1:length(valuable_and_heavy)
    add_to_knapsack(valuable_and_heavy[i], J, knapsack, list_of_possible_corners)
end

10NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 10, dim = (ai = 10, bi = 3, ci = 5))]NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3))]NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5))]

**Output:** 

In [31]:
J

7-element Array{Tuple{NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}},NamedTuple{(:xi, :yi, :zi),Tuple{Int64,Int64,Int64}},NamedTuple{(:hi, :wi),Tuple{String,String}}},1}:
 ((vi = 0, dim = (ai = 0, bi = 0, ci = 0)), (xi = 0, yi = 0, zi = 0), (hi = "ai", wi = "bi"))   
 ((vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (xi = 0, yi = 0, zi = 0), (hi = "bi", wi = "ai")) 
 ((vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (xi = 10, yi = 0, zi = 0), (hi = "bi", wi = "ci"))
 ((vi = 30, dim = (ai = 1, bi = 3, ci = 5)), (xi = 0, yi = 5, zi = 0), (hi = "bi", wi = "ci"))  
 ((vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (xi = 0, yi = 0, zi = 3), (hi = "ci", wi = "ai")) 
 ((vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (xi = 10, yi = 3, zi = 0), (hi = "bi", wi = "ai")) 
 ((vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (xi = 10, yi = 0, zi = 3), (hi = "ci", wi = "ai")) 

In [32]:
V = calculate_V_in_J(J)

345

In [33]:
#- Strategy 1 [Putting Rock first then later Sand]
#    Step1: Order them based on Weight
items_bigger_to_smaller =  order_items_by(items, "volume")
#    Step2: Divide them in half
bigger_half, smaller_half = split_in_halves(items_bigger_to_smaller)
#    Step3: Order each half based on Value
bigger_and_valuable = order_items_by(bigger_half, "value")
smaller_and_valuable = order_items_by(smaller_half, "value")
#    Step4: Take the heaviest and most valuable until it all fits
for i in 1:length(bigger_and_valuable)
    list_of_possible_corners = calculate_new_corners(J, knapsack)
    add_to_knapsack(bigger_and_valuable[i], J, knapsack, list_of_possible_corners)
end
#    Step5: Take the lightest and most valuable if there is enough space left
for i in 1:length(smaller_and_valuable)
    list_of_possible_corners = calculate_new_corners(J, knapsack)
    add_to_knapsack(smaller_and_valuable[i], J, knapsack, list_of_possible_corners)
end

NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 10, dim = (ai = 10, bi = 3, ci = 5)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5))]10NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 105, dim = (ai = 3, bi = 3, ci = 3)), (vi = 20, dim = (ai = 3, bi = 3, ci = 10)), (vi = 10, dim = (ai = 10, bi = 3, ci = 5))]95NamedTuple{(:vi, :dim),Tuple{Int64,NamedTuple{(:ai, :bi, :ci),Tuple{Int64,Int64,Int64}}}}[(vi = 95, dim = (ai = 3, bi = 3, ci = 3)), (vi = 85, dim = (ai = 3, bi = 3, ci = 3)), (vi = 30, dim = (ai = 1, bi = 3, ci = 5))]

UndefVarError: UndefVarError: calculate_new_corners not defined

In [ ]:
J